# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  * Fernando Omar Salazar Ortiz - A01796214
  * Carlos Aaron Bocanegra Buitron - A01796345
  * Luis Enrique González González - A01795338
  * Gloria María Campos García - A01422345

* **Número de Equipo:**


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

In [8]:
#!pip install -q openai
!pip install -q openai PyPDF2 sentence-transformers faiss-cpu


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [129]:
#Se hacen las importaciones necesarias
from google.colab import userdata
import openai
import PyPDF2
import tiktoken
import numpy as np
import faiss
import pickle

from sentence_transformers import SentenceTransformer

In [130]:
"""
Guarda los vectores (embeddings) y sus textos (chunks) en un archivo .pkl.

Args:
    vectors (List[List[float]] o np.ndarray): Embeddings generados.
    chunks (List[str]): Lista de textos originales de cada embedding.
    file_path (str): Ruta de archivo donde se guardará todo.
"""
def save_embeddings_to_pkl(vectors, chunks, file_path="embeddings_data.pkl"):
    data = {
        "vectors": np.array(vectors).astype("float64"),
        "chunks": chunks
    }
    with open(file_path, "wb") as f:
        pickle.dump(data, f)
    print(f"✅ Embeddings guardados en {file_path}")


"""
Carga los vectores (embeddings) y los textos (chunks) desde un archivo .pkl.

Returns:
    Tuple[np.ndarray, List[str]]: embeddings y chunks.
"""
def load_embeddings_from_pkl(file_path="embeddings_data.pkl"):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    print(f"📂 Cargados {len(data['chunks'])} embeddings desde {file_path}")
    return data["vectors"], data["chunks"]


In [9]:
# Se carga la API Key de OpenAI


api_key = userdata.get("OPENAI_API_KEY_PERSONAL")



if not api_key:
  raise ValueError("API key no encontrada en los secretos")


client = openai.OpenAI(api_key=api_key)

In [138]:
# 1. EXTRAER TEXTO DEL PDF
def extract_text_from_pdf(pdf_path, skip_pages=None):
    if skip_pages is None:
        skip_pages = []

    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""

        for i, page in enumerate(reader.pages):
            if i in skip_pages:
                continue  # Salta la página
            text += page.extract_text() + "\n"

    return text

# 2. DIVIDIR EN CHUNKS DE ~500 TOKENS
def chunk_text(text, max_tokens=500, overlap=100):
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = enc.decode(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap
    return chunks

# 3. GENERAR EMBEDDINGS 
def embed_texts(texts):
    embeddings = []
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts)
    return np.array(embeddings).astype("float32")


def embed_texts_openai(texts):
    embeddings = []
    for chunk in texts:
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-3-small"
        )
        embedding = response.data[0].embedding
        embeddings.append(embedding)
    return np.array(embeddings).astype("float32")

# 4. Crear indice FAISS
def create_faiss_index(embeddings):
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# 4. RAG: RECUPERAR Y GENERAR RESPUESTA
def retrieve_and_answer(question, chunks, index, chunk_embeddings, top_k=3):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    q_embedding = model.encode(question)
     
    q_vector = np.array(q_embedding).astype("float32").reshape(1, -1)
    D, I = index.search(q_vector, top_k)
    print("------------------------------------")
    for rank, idx in enumerate(I[0]):
        print(f"\n--- Chunk #{idx} --")
        print(f"Distancia (D): {D[0][rank]} \n")
        print(chunks[idx])
        

    print("------------------------------------")
    context = "\n---\n".join([chunks[i] for i in I[0]])
    '''
    prompt = f"""Usa la siguiente información para responder la pregunta de forma clara y completa.
Información:
{context}

Pregunta: {question}
Respuesta:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content
    '''
    return "TEST"


def retrieve_and_answer_openai(question, chunks, index, chunk_embeddings, top_k=3):
    q_embedding = client.embeddings.create(
                                            input=question,
                                            model="text-embedding-3-small"
                                        ).data[0].embedding
    q_vector = np.array(q_embedding).astype("float32").reshape(1, -1)
    D, I = index.search(q_vector, top_k)
    print("------------------------------------")
    for rank, idx in enumerate(I[0]):
        print(f"--- Chunk #{idx} --")
        print(f"Distancia (D): {D[0][rank]} \n")
        print(chunks[idx])
    print("------------------------------------")
    return "TEST"

In [118]:
# 5. FLUJO COMPLETO
pdf_path = "lwc-pregnancy-guide_spanish-1.pdf"  # Reemplaza con tu archivo
text = extract_text_from_pdf(pdf_path, skip_pages=[0, 1, 22, 23, 24])

In [119]:
chunks = chunk_text(text, max_tokens=200, overlap=25)

In [124]:
#embeddings = embed_texts(chunks)
embeddings = embed_texts_openai(chunks)

In [125]:
faiss_index = create_faiss_index(embeddings)

In [140]:
#pregunta = "¿Se puede consumir cafeína durante el embarazo?"
#pregunta = "¿Qué son las vitaminas prenatales?"
#pregunta = "¿Todas las mujeres deben de tomar vitaminas prenatales durante el embarazo?"
#pregunta = "¿Qué frutas puedo comer durante el embarazo?" # No tiene buenos resultados esta pregunta
pregunta = "¿Se puede viajar durante el embarazo?"
#respuesta = retrieve_and_answer(pregunta, chunks, faiss_index, embeddings)
respuesta = retrieve_and_answer_openai(pregunta, chunks, faiss_index, embeddings)
print("\n🤖 Respuesta del chatbot:\n", respuesta)

------------------------------------
--- Chunk #37 --
Distancia (D): 0.7017810344696045 

 después del sexo, puede ser útil usar 
condón. El semen humano contiene hormonas que pueden causar contracciones. La mayoría de las parejas tendrán cambios en 
sus patrones sexuales o libido durante el embarazo. Más que nunca, es importante que haya comprensión mutua y comunicación 
abierta. Puede que usted necesite explorar métodos nuevos para complacerse el uno al otro. La mujer embarazada puede tener 
un orgasmo sin peligro. 
Viajar
Su mejor guía será el sentido común. Si va a viajar una distancia larga, se encontrará más cómoda si usted se permite estirar, 
caminar, y vaciar la vejiga cada dos horas. Está bien viajar en avión, pero algunas aerolíneas exigen un permiso escrito por su 
doctor si viaja durante el último trimestre. Durante el último mes de
--- Chunk #38 --
Distancia (D): 0.7457499504089355 

igen un permiso escrito por su 
doctor si viaja durante el último trimestre. Durante el ú

In [132]:
len(embeddings)

105

In [134]:
len(chunks)

105

In [135]:
save_embeddings_to_pkl(embeddings, chunks, file_path='embeddings_test_lwc_pdf.pkl')

✅ Embeddings guardados en embeddings_test_lwc_pdf.pkl


In [136]:
vectors, texts =load_embeddings_from_pkl(file_path='embeddings_test_lwc_pdf.pkl')

📂 Cargados 105 embeddings desde embeddings_test_lwc_pdf.pkl


In [77]:
#import numpy as np

def l2_distance(a, b):
    return np.linalg.norm(a - b)

# Comparar con, por ejemplo, el chunk #17
question = "¿Qué son las vitaminas prenatales?"
model = SentenceTransformer('all-MiniLM-L6-v2')
q_embedding = model.encode(question)

distance = l2_distance(q_embedding, embeddings[27])
print(f"Distancia: {distance}")


Distancia: 1.0078504085540771


# **Conclusiones:**

* #### **Incluyan sus conclusiones de la actividad chatbot LLM + RAG:**



None

# **Fin de la actividad chatbot: LLM + RAG**